# Prediciendo la diabetes

In [2]:
import pandas as pd

url_test = "https://raw.githubusercontent.com/DalynLC/RandomForest/main/data/processed/diabetes_test.csv"
url_train = "https://raw.githubusercontent.com/DalynLC/RandomForest/main/data/processed/diabetes_train.csv"

df_test = pd.read_csv(url_test)
df_train = pd.read_csv(url_train)

df_train.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.058824,0.419355,0.142857,0.012195,0.123188,0.100204,0.140478,0.083333,0.0
1,0.000000,0.361290,0.469388,0.109756,0.038647,0.257669,0.221605,0.000000,0.0
2,0.000000,0.877419,0.428571,0.268293,0.074879,0.486708,0.774979,0.066667,1.0
3,0.235294,0.548387,0.632653,0.036585,0.304348,0.345603,0.065329,0.033333,0.0
4,0.117647,0.258065,0.265306,0.073171,0.070048,0.249489,0.380017,0.000000,0.0


In [5]:
# Separar características (X) y etiquetas (y)
X_train = df_train.drop(["Outcome"], axis=1)
y_train = df_train["Outcome"]
X_test = df_test.drop(["Outcome"], axis=1)
y_test = df_test["Outcome"]

print(X_train.head())
print(y_train.head())

   Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
0     0.058824  0.419355       0.142857       0.012195  0.123188  0.100204   
1     0.000000  0.361290       0.469388       0.109756  0.038647  0.257669   
2     0.000000  0.877419       0.428571       0.268293  0.074879  0.486708   
3     0.235294  0.548387       0.632653       0.036585  0.304348  0.345603   
4     0.117647  0.258065       0.265306       0.073171  0.070048  0.249489   

   DiabetesPedigreeFunction       Age  
0                  0.140478  0.083333  
1                  0.221605  0.000000  
2                  0.774979  0.066667  
3                  0.065329  0.033333  
4                  0.380017  0.000000  
0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: Outcome, dtype: float64


In [10]:
# Entrenar el modelo con XGBoost
from xgboost import XGBClassifier

model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:19:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [11]:
# Realizar predicciones
y_pred = model.predict(X_test)
print(y_pred)

[1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1
 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1
 0 0 1]


In [13]:
from sklearn.metrics import accuracy_score
# Calcular y mostrar precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 0.7483


La precisión del modelo obtenida, 0.7483, indica que aproximadamente el 74.83% de las predicciones realizadas por el clasificador XGBoost coinciden con los valores reales del conjunto de prueba. Este resultado sugiere un desempeño aceptable del modelo, aunque deja un margen importante de error (alrededor del 25%).

In [15]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
import xgboost as xgb

# Definir distribuciones de hiperparámetros
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators': stats.randint(50, 200)
}

# Crear el modelo base de XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Crear objeto RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42
)

# Ajustar búsqueda a los datos de entrenamiento
random_search.fit(X_train, y_train)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros encontrados: ", random_search.best_params_)

# Evaluar el mejor modelo
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy


/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/trainin

Mejores hiperparámetros encontrados:  {'learning_rate': np.float64(0.015808361216819947), 'max_depth': 7, 'n_estimators': 149, 'subsample': np.float64(0.5714334089609704)}


/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:25:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [16]:
from xgboost import XGBClassifier

# Crear el modelo con los hiperparámetros especificados
model = XGBClassifier(
    random_state=42,
    learning_rate=0.07118528947223794,
    max_depth=4,
    n_estimators=64,
    subsample=0.728034992108518
)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07118528947223794, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=64, n_jobs=None,
              num_parallel_tree=None, ...)

In [18]:
# Realizar predicciones
y_pred = model.predict(X_test)

y_pred

array([1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1])

In [19]:
from sklearn.metrics import accuracy_score

# Calcular y mostrar la precisión
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.8013245033112583

La precisión del modelo de 0.8013 indica que aproximadamente el 80.13% de las predicciones realizadas por el modelo son correctas, lo cual sugiere un rendimiento bastante sólido en la clasificación de los datos de prueba. Este nivel de precisión es relativamente alto, lo que implica que el modelo ha logrado identificar correctamente la mayoría de los casos, aunque sigue habiendo un margen de error (cerca del 20%) que podría ser crítico dependiendo del contexto de aplicación, como en un diagnóstico médico. 

In [22]:
from pickle import dump

# Almacenamiento del modelo en un archivo .sav
dump(model, open("../models/boosting_classifier_nestimators-20_learnrate-0.001_42.sav", "wb"))
